# **Amazon Reviews를 이용한 추천시스템 실습 - surprise패키지 사용**

In [ ]:
import numpy as np 
import pandas as pd
import os
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import math
import json
import time
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.model_selection import train_test_split
from sklearn.neighbors import NearestNeighbors
# from sklearn.externals import joblib
import scipy.sparse
from scipy.sparse import csr_matrix
from scipy.sparse.linalg import svds
import warnings; warnings.simplefilter('ignore')
%matplotlib inline

In [ ]:
from google.colab import drive 
drive.mount('/content/drive/')

Mounted at /content/drive/


In [ ]:
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

electronics_data = pd.read_csv("/content/drive/MyDrive/ESAA-OB/0516/ratings_Electronics (1).csv",names=['userId', 'productId','Rating','timestamp'])

In [ ]:
eletronics_data = electronics_data.iloc[:1048576,0:]

### - Missing values 수

In [ ]:
eletronics_data.drop(['timestamp'],axis = 1, inplace = True)

### - Ratings 분석

### - Popularity Based Recommendation

In [ ]:
new_df=electronics_data.groupby("productId").filter(lambda x:x['Rating'].count() >=50)

## **Collaborative Filtering**

In [ ]:
!pip3 install surprise


     |████████████████████████████████| 11.8 MB 5.0 MB/s 
  Created wheel for scikit-surprise: filename=scikit_surprise-1.1.1-cp37-cp37m-linux_x86_64.whl size=1633716 sha256=bdf0236bb41f75f8995a531efcb3d25d46f96ad9b37bbf55167045ab89864f20
  Stored in directory: /root/.cache/pip/wheels/76/44/74/b498c42be47b2406bd27994e16c5188e337c657025ab400c1c
Successfully built scikit-surprise


In [ ]:
from surprise import KNNWithMeans
from surprise import Dataset
from surprise import accuracy
from surprise import Reader
import os
from surprise.model_selection import train_test_split

In [ ]:
gpu_info = !nvidia-smi
gpu_info = '\n'.join(gpu_info)
if gpu_info.find('failed') >= 0:
  print('Not connected to a GPU')
else:
  print(gpu_info)

Mon May 23 10:19:08 2022       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.32.03    Driver Version: 460.32.03    CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  Tesla T4            Off  | 00000000:00:04.0 Off |                    0 |
| N/A   43C    P8     9W /  70W |      0MiB / 15109MiB |      0%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [ ]:
new_df.drop(['timestamp'],axis = 1, inplace = True)

In [ ]:
reader = Reader(rating_scale=(1,5))
data = Dataset.load_from_df(new_df,reader)

In [ ]:
trainset, testset = train_test_split(data, test_size = 0.3, random_state=10)

In [ ]:
algo = KNNWithMeans(k=5, sim_options = {'name': 'pearson_baseline', 'user_based':False})
algo.fit(trainset)

Estimating biases using als...
Computing the pearson_baseline similarity matrix...


In [ ]:
test_pred = algo.test(testset)

NameError: ignored

In [ ]:
print("Item-based Model : Test Set")
accuracy.rmse(test_pred, verbose=True)

Model-based collaborative filtering system

In [ ]:
new_df1 = new_df.head(10000)
ratings_matrix = new_df1.pivot_table(values ='Rating', index ='userId', columns ='productId',fill_value =0)
ratings_matrix.head()

In [ ]:
ratings_matrix.shape

In [ ]:
X = ratings_matrix.T
X.head()

In [ ]:
X.shape

In [ ]:
X.index

In [ ]:
X1 = X

In [ ]:
from sklearn.decomposition import TruncatedSVD
SVD = TruncatedSVD(n_components=10)
decomposed_matrix = SVD.fit_transform(X)
decomposed_matrix.shape

In [ ]:
decomposed_matrix

In [ ]:
correlation_matrix = np.corrcoef(decomposed_matrix)
correlation_matrix.shape

In [ ]:
X.index[75]

In [ ]:
i = X.index[75]

product_names = list(X.index)
product_ID = product_names.index(i)
product_ID

In [ ]:
correlation_product_ID = correlation_matrix[product_ID]
correlation_product_ID.shape

In [ ]:
correlation_product_ID

In [ ]:
Recommend = list(X.index[correlation_product_ID > 0.65])

Recommend.remove(i)
Recommend[0:24]